In [1]:
pip install openai -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00


In [2]:
import openai
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import json

In [14]:
def get_start_time(start_datetime_script):
    local_vars = {}
    start_datetime_script = f'''
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
''' + start_datetime_script
    exec(start_datetime_script, {}, local_vars)
    if 'start_datetime' in local_vars:
        return local_vars['start_datetime']
    elif 'end_datetime' in local_vars:  # Sometimes the llm get confused with the variable name
        return local_vars['end_datetime']
    else:
        raise ValueError("The script did not assign result to a variable")


def get_end_time(end_datetime_script):
    local_vars = {}
    end_datetime_script = f'''
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
''' + end_datetime_script
    exec(end_datetime_script, {}, local_vars)
    if 'end_datetime' in local_vars:
        return local_vars['end_datetime']
    elif 'start_datetime' in local_vars:    # Sometimes the llm get confused with the variable name
        return local_vars['start_datetime']
    else:
        raise ValueError("The script did not assign result to a variable")


# Get API key from environment variable
api_key = ""

if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

client = openai.OpenAI(api_key=api_key)

tools = [{
    "type": "function",
    "function": {
        "name": "get_start_time",
        "description": f"""
        This function takes a string of python script as arguments. The python script, when executed using exec(), make a `datetime` object corresponds to the start of the time, period or date that the user described.
        The datetime object must be assigned to a local variable named 'start_datetime'.
        The function returns a datetime object 'start_datetime' describing the time the user is referring to.""",
        "parameters": {
            "type": "object",
            "properties": {
                "start_datetime_script": {
                    "type": "string",
                    "description": f"""
                    A string of Python script that, when executed using exec(), generates a `datetime` object representing the start of the time that the user described.
                    The datetime object must be assigned to a local variable named 'start_datetime'.
                    The script can utilize `datetime` and `timedelta` from the `datetime` module, and `relativedelta` from `dateutil.relativedelta`.
                    The script should directly assign the `datetime` expression to 'start_datetime', such as 'start_datetime = datetime(year=2024, month=8, day=23, hour=10, minute=30, second=0)`.""",
                }
            },
            "required": ["start_datetime_script"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "get_end_time",
        "description": f"""
        This function takes a string of python script as arguments. The python script, when executed using exec(), make a `datetime` object corresponds to the end of the time, period or date that the user described.
        The datetime object must be assigned to a local variable named 'end_datetime'.
        The function returns a datetime object 'end_datetime' describing the time the user is referring to.""",
        "parameters": {
            "type": "object",
            "properties": {
                "end_datetime_script": {
                    "type": "string",
                    "description": f"""
                    A string of Python script that, when executed using exec(), generates a `datetime` object representing the end of the time that the user described.
                    The datetime object must be assigned to a local variable named 'end_datetime'.
                    The script can utilize `datetime` and `timedelta` from the `datetime` module, and `relativedelta` from `dateutil.relativedelta`.
                    The script should directly assign the `datetime` expression to 'end_datetime', such as 'end_datetime = datetime(year=2024, month=8, day=23, hour=10, minute=30, second=0)`.""",
                }
            },
            "required": ["end_datetime_script"]
        }
    }
}
]
def get_time_from_question(question):
    # First, use the is_time tool to check if the question refers to time
    messages = [{"role": "user", "content": question}]
    completion = client.chat.completions.create(
        # model="gpt-3.5-turbo",
        model="gpt-4o",
        messages=messages,
        tools=tools)

    # Check the response from is_time
    if completion.choices[0].message.tool_calls:
        tool_calls = completion.choices[0].message.tool_calls
        calls = {}
        for call in tool_calls:
            calls[call.function.name] = call
        # print(calls)
        if "get_start_time" not in calls and "get_end_time" not in calls:
            now = datetime.now()
            return f"The user is talking about time from {now - timedelta(hours=1)} up to {now}."
        elif "get_start_time" not in calls:
            return "You must specify a from_datetime."
        elif "get_end_time" not in calls:
            now = datetime.now()
            tool_call = calls.get("get_start_time")
            arguments_json = tool_call.function.arguments
            arguments = json.loads(arguments_json)
            start_time = get_start_time(**arguments)
            return f"The user is talking about time from {start_time} up to {now}."
        else:  # ("get_start_time" in calls and "get_end_time" in calls:)
            tool_call = calls.get("get_start_time")
            arguments_json = tool_call.function.arguments
            arguments = json.loads(arguments_json)
            start_time = get_start_time(**arguments)

            tool_call = calls.get("get_end_time")
            arguments_json = tool_call.function.arguments
            arguments = json.loads(arguments_json)
            end_time = get_end_time(**arguments)
            return f"The user is talking about time from {start_time} up to {end_time}."

    else:
        now = datetime.now()
        return f"The user is talking about time from {now - timedelta(hours=1)} up to {now}."

### Easy - Medium time questions

In [15]:
questions=[
          "where did you go from 15.08.2023 to 20 Oct 2023",
          "where did you go from August 15 2023 to 20.10.2023?",
          "where did you go between two months ago until a month ago?",
          "where did you go two months ago?",
          "where did you go until 15/08/2023?",
          "where is Argentina located?",
          "what is the highest mountain in the world?",
           ]
for question in questions:
  print(f"Question: {question}\n")
  print(get_time_from_question(question))
  print("------------------------------------------------")

Question: where did you go from 15.08.2023 to 20 Oct 2023

The user is talking about time from 2023-08-15 00:00:00 up to 2023-10-20 00:00:00.
------------------------------------------------
Question: where did you go from August 15 2023 to 20.10.2023?

The user is talking about time from 2023-08-15 00:00:00 up to 2023-10-20 00:00:00.
------------------------------------------------
Question: where did you go between two months ago until a month ago?

The user is talking about time from 2024-06-24 10:20:17.525636 up to 2024-08-24 10:20:17.525481.
------------------------------------------------
Question: where did you go two months ago?

The user is talking about time from 2024-06-25 10:20:18.161474 up to 2024-08-24 10:20:18.161303.
------------------------------------------------
Question: where did you go until 15/08/2023?

You must specify a from_datetime.
------------------------------------------------
Question: where is Argentina located?

The user is talking about time from 2024

In [17]:
questions=[
            "לאן הלכת מה15.08.2023 ועד ה20 באוקטובר 2023?",
            "לאן הלכת מהחמש עשרה באוגוסט 2023 ועד ה20.10.2023?",
            "לאן הלכת לפני חודשיים ועד לפני חודש?",
            "לאן הלכת לפני חודשיים?",
            "לאן הלכת עד ה15.08.2023?",
            "איפה ממוקמת ארגנטינה?",
            "מה ההר הכי גבוה בעולם?",
           ]
for question in questions:
  print(f"Question: {question}\n")
  print(get_time_from_question(question))
  print("------------------------------------------------")

Question: לאן הלכת מה15.08.2023 ועד ה20 באוקטובר 2023?

The user is talking about time from 2023-08-15 00:00:00 up to 2023-10-20 00:00:00.
------------------------------------------------
Question: לאן הלכת מהחמש עשרה באוגוסט 2023 ועד ה20.10.2023?

The user is talking about time from 2023-08-15 00:00:00 up to 2023-10-20 00:00:00.
------------------------------------------------
Question: לאן הלכת לפני חודשיים ועד לפני חודש?

The user is talking about time from 2024-06-24 10:21:02.564222 up to 2024-07-24 10:21:02.564369.
------------------------------------------------
Question: לאן הלכת לפני חודשיים?

The user is talking about time from 2024-06-24 10:21:03.176354 up to 2024-08-24 10:21:03.176233.
------------------------------------------------
Question: לאן הלכת עד ה15.08.2023?

You must specify a from_datetime.
------------------------------------------------
Question: איפה ממוקמת ארגנטינה?

The user is talking about time from 2024-08-24 09:21:04.878073 up to 2024-08-24 10:21:04.8780

### Hard time questions

In [18]:
questions=[
            "where did you go between Passover 2022 to the begining of Christmas 2023",
            "where did you go 3 days before the longest day of the year until the date of the end of the second quarter",
            "where did you go between the death date of Amy Winehouse until the release of 1989 album of Taylor Swift",
           ]
for question in questions:
  print(f"Question: {question}\n")
  print(get_time_from_question(question))
  print("------------------------------------------------")

Question: where did you go between Passover 2022 to the begining of Christmas 2023

The user is talking about time from 2022-04-15 00:00:00 up to 2023-12-24 00:00:00.
------------------------------------------------
Question: where did you go 3 days before the longest day of the year until the date of the end of the second quarter

The user is talking about time from 2024-06-18 00:00:00 up to 2024-06-30 00:00:00.
------------------------------------------------
Question: where did you go between the death date of Amy Winehouse until the release of 1989 album of Taylor Swift

The user is talking about time from 2011-07-23 00:00:00 up to 2014-10-27 00:00:00.
------------------------------------------------


In [19]:
questions=[
            "לאן הלכת מפסח 2022 ועד לתחילת חג המולד 2023?",
            "לאן הלכת 3 ימים לפני היום הכי ארוך בשנה ועד לסוף הרבעון השני?",
            "לאן הלכת בין תאריך פטירתה של איימי ויינהאוס לבין הוצאת האלבום 1989 של טיילור סוויפט?",
           ]
for question in questions:
  print(f"Question: {question}\n")
  print(get_time_from_question(question))
  print("------------------------------------------------")

Question: לאן הלכת מפסח 2022 ועד לתחילת חג המולד 2023?

The user is talking about time from 2022-04-15 00:00:00 up to 2023-12-25 00:00:00.
------------------------------------------------
Question: לאן הלכת 3 ימים לפני היום הכי ארוך בשנה ועד לסוף הרבעון השני?

The user is talking about time from 2024-06-18 00:00:00 up to 2024-06-30 23:59:59.
------------------------------------------------
Question: לאן הלכת בין תאריך פטירתה של איימי ויינהאוס לבין הוצאת האלבום 1989 של טיילור סוויפט?

The user is talking about time from 2011-07-23 00:00:00 up to 2014-10-27 00:00:00.
------------------------------------------------
